In [1]:
# load the library
import numpy as np
import pandas as pd
from tqdm import tqdm

# load selenium
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pyperclip
from IPython.display import clear_output
import sqlite3
from pandas.io import sql
import datetime

In [2]:
# handling page refresh
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

In [48]:
# Set name of your SQLite database
db_name = "web_freight_data"

sqlite_db = db_name + '.sqlite'
conn = sqlite3.connect(sqlite_db)
c = conn.cursor()

In [4]:
# load the library
import requests
import bs4
from bs4 import BeautifulSoup
import numpy as np
import requests
import pandas as pd
from tqdm import tqdm
pd.set_option('max_colwidth',1000)
import time as time_
from sklearn.externals import joblib

In [19]:
# download chromedriver.exe and save to a local drive locaiton:
location_of_chromedriver = "Driver/chromedriver.exe"
site_url = "http://www.worldfreightrates.com/en/freight"

In [21]:
# set browser and go to google maps
def init_browser(URL = 'https://www.google.com.hk/maps'):
    browser = webdriver.Chrome(location_of_chromedriver)
    browser.get(URL)
    return browser

In [7]:
def page_wait(look_for, max_wait_time = 20):
    success = False
    counter = 0
    while success == False:
        counter = counter + 1
        time_.sleep(1)
        tmp_texts = rate_site.find_elements_by_class_name(look_for)
        for i in range(len(tmp_texts)):
            try:
                tmp_text = tmp_texts[i].text
                if tmp_text != '':
                    success = True
            except:
                pass
        if counter > max_wait_time:
            success = True
    return counter

In [8]:
def handle_output(output_text):
    # split elements of details
    tmp = output_text
    detail_ele = tmp.split('\n')
    dist = detail_ele[0]
    times = detail_ele[-1]
    
    if dist.find('km') != -1:
        mile = dist[:dist.find('mi')]
        km = dist[dist.find('mi')+3:]
        mile = mile.strip(" ")
        mile = float(mile)
        km = km.strip('(')
        km = km.strip(')')
        km = km.strip(" ")
        km = km.strip('km')
        km = km.strip(' ')
        km = float(km)
    #     print(mile)
    #     print(km)
    else:
        mile = 0
        km = 0
    
    if times.find('Transit Time:') != -1:
        duration = times.strip(' ')
        duration = duration.replace('Transit Time: ','')
        duration = duration.split(' ')
        time = duration[0]
        time_uom = duration[1]
    else:
        time = 0
        time_uom = ""
    
    return mile, km, time, time_uom

In [32]:
def search_route(orig_port, dest_port, mode, equip, wait_time = 30):
#     orig_port = 'Hong Kong'
#     dest_port = 'Tokyo'
#     wait_time = 30
#     mode = TRUCK # Air # Ocean
#     equip = '20FT'

    # apply search
    if mode == 'TRUCK':
        ele_name = 'fromCity'
    else:
        ele_name = 'fromName{}'.format(mode)
    orig_input = rate_site.find_element_by_id(ele_name)
    orig_input.clear()
    orig_input.send_keys(orig_port)
    page_wait('ui-menu-item', wait_time)
    orig_input.send_keys(Keys.DOWN)
    orig_input.send_keys(Keys.ENTER)
    
    if mode == 'TRUCK':
        ele_name = 'toCity'
    else:
        ele_name = 'toName{}'.format(mode)
    dest_input = rate_site.find_element_by_id(ele_name)
    dest_input.clear()
    dest_input.send_keys(dest_port)
    page_wait('ui-menu-item', wait_time)
    dest_input.send_keys(Keys.DOWN)
    dest_input.send_keys(Keys.ENTER)

    w_count = 0
    success = False
    keep = -1
    time_.sleep(1)
    while success != True:
        search_btn = rate_site.find_elements_by_class_name('submit')
        for i in range(len(search_btn)):
            if search_btn[i].text.upper() == "GET RATE":
                success = True
                keep = i
        if w_count > wait_time:
            success = True
        w_count = w_count + 1
        time_.sleep(1)
    time_.sleep(1)
    search_btn[keep].click()

    page_wait('result', wait_time)
    try:
        segment_summary = rate_site.find_elements_by_class_name('result')
        result_rate = segment_summary[-1].text
    except:
        result_rate = 0
    orig_txt = orig_input.get_attribute("value")
    dest_txt = dest_input.get_attribute("value")


    # setup return sets
    orig_set = orig_port
    dest_set = dest_port
    price_set = result_rate
    mode_set = mode
    equip_set = equip
    # build dataframe
    summary_df = pd.DataFrame()
    summary_df['Orig'] = [orig_set]
    summary_df['Dest'] = dest_set
    summary_df['Orig_txt'] = orig_txt
    summary_df['Dest_txt'] = dest_txt
    summary_df['Mode'] = mode_set
    summary_df['Equipment'] = equip_set
    summary_df['Rate'] = result_rate 
    now = datetime.datetime.now()
    now = now.strftime("%Y-%m-%d %H:%M")
    summary_df['extraction_time'] = now

#     summary_df.to_sql('world_freight_rate', con=conn, index=False, if_exists='append')

    return summary_df

In [22]:
rate_site = init_browser(site_url)

In [11]:
# if_name = rate_site.find_element_by_tag_name("iframe")
# rate_site.switch_to_frame(if_name)

# switch back to main page
# rate_site.switch_to_default_content()

In [37]:
counter = 0

In [43]:
freight_lanes = pd.read_excel('freight_lanes.xlsx')

In [44]:
len(freight_lanes)

30

In [52]:
search_route(orig_port, dest_port, mode = 'Ocean', equip = '40FT')

,Orig,Dest,Orig_txt,Dest_txt,Mode,Equipment,Rate,extraction_time
0,YANTIAN,SOUTHAMPTON,"Yantian, China","Southampton, United Kingdom",Ocean,40FT,"$1,907.63 - $2,108.43",2018-06-24 10:51


In [ ]:
# Run while loop to get rates
lanes_df = pd.DataFrame()

while counter < len(freight_lanes):

# for i in tqdm(range(len(freight_lanes))):
    print(counter)
    i = counter
    orig_port = freight_lanes['ORIG_PORT'].iloc[i]
    dest_port = freight_lanes['DEST_PORT'].iloc[i]
    summary_df = search_route(orig_port, dest_port, mode = 'Ocean', equip = '40FT')
    
    lanes_df = lanes_df.append(summary_df)
    counter = counter + 1
    clear_output()
print(counter)

21


In [39]:
lanes_df.to_excel('lane_cost.xlsx',index=False)

,Orig,Dest,Orig_txt,Dest_txt,Mode,Equipment,Rate,extraction_time
0,Jakarta,Hong Kong,"Jakarta, Indonesia","Kwai Chung, Hong Kong",Ocean,40FT,$447.19 - $494.26,2018-06-24 10:37


In [27]:
web_freight_rates = pd.read_sql('select * from world_freight_rate', con = conn)
web_freight_rates.drop_duplicates(inplace=True)
web_freight_rates.to_csv('lanes_summary.csv', index=False, encoding='utf_8')

In [ ]:
rate_site.close()
# conn.close()